In [1]:
print("Hello world!")

Hello world!


In [13]:
import os
import re
import joblib
import random
import shutil
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
random.seed(42)
np.random.seed(42)

In [4]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
price_df = pd.read_csv("/content/drive/MyDrive/DataCrunch/FMCG/PriceData/train_data.csv")

price_df.head(10)

,Date,Region,Commodity,Price per Unit (Silver Drachma/kg),Type
0,2040-01-01,Mystic Falls,Plantain,296.37,Fruit
1,2040-01-01,Olympus,Loquat,394.21,Fruit
2,2040-01-01,Metropolis,Loquat,308.13,Fruit
3,2040-01-01,Arcadia,Loquat,460.81,Fruit
4,2040-01-01,Elysium,Loquat,85.58,Fruit
5,2040-01-01,Shangri-La,Loquat,291.70,Fruit
6,2040-01-01,Valhalla,Cantaloupe,471.78,Fruit
7,2040-01-01,Eldoria,Loquat,335.06,Fruit
8,2040-01-01,Valhalla,Loquat,307.71,Fruit
9,2040-01-01,El Dorado,Loquat,314.78,Fruit


In [19]:
price_df.tail(10)

,Date,Region,Commodity,Price per Unit (Silver Drachma/kg),Type
181720,2043-06-30,Avalon,Butternut Squash,352.81,Vegetable
181721,2043-06-30,Olympus,Butternut Squash,238.79,Vegetable
181722,2043-06-30,Metropolis,Butternut Squash,193.37,Vegetable
181723,2043-06-30,Arcadia,Butternut Squash,304.89,Vegetable
181724,2043-06-30,Elysium,Butternut Squash,493.13,Vegetable
181725,2043-06-30,Shangri-La,Butternut Squash,189.08,Vegetable
181726,2043-06-30,Asgard,Sweet Potato,387.87,Vegetable
181727,2043-06-30,Rivendell,Butternut Squash,320.00,Vegetable
181728,2043-06-30,Winterfell,Butternut Squash,360.21,Vegetable
181729,2043-06-30,Gotham,Butternut Squash,59.46,Vegetable


In [15]:
weather_df = pd.read_csv("/content/drive/MyDrive/DataCrunch/FMCG/WeatherData/train_data.csv")

weather_df.head(10)

,Date,Region,Temperature (K),Rainfall (mm),Humidity (%),Crop Yield Impact Score
0,2040-01-01,Mystic Falls,307.55,474.5,61.3,1.89
1,2040-01-01,Olympus,301.35,106.6,53.6,1.74
2,2040-01-01,Metropolis,306.15,266.3,93.9,2.18
3,2040-01-01,Arcadia,304.15,222.2,81.0,1.68
4,2040-01-01,Elysium,304.85,492.1,61.7,1.89
5,2040-01-01,Shangri-La,295.45,404.4,86.0,0.98
6,2040-01-01,Valhalla,302.65,492.9,89.4,1.22
7,2040-01-01,Eldoria,305.55,81.3,74.6,2.22
8,2040-01-01,Valhalla,302.65,492.9,89.4,1.22
9,2040-01-01,El Dorado,310.35,203.1,79.8,2.05


In [9]:
eval_price_df = pd.read_csv("/content/drive/MyDrive/DataCrunch/FMCG/PriceData/eval_data.csv")

eval_price_df.shape

(26184, 5)

In [6]:
# Generalized helper
def prepare_dataset(df, region, typ, le=None, fit_encoder=False):
    df = (
        df.query("Region == @region and Type == @typ")
          .assign(Date=lambda d: pd.to_datetime(d['Date']))
          .sort_values(['Commodity','Date'])
    )
    if fit_encoder:
        le = LabelEncoder().fit(df['Commodity'])
    df['commodity_enc'] = le.transform(df['Commodity'])

    # build lags & leads
    for lag in [1,2,3,4]:
        df[f'lag_{lag}'] = df.groupby('Commodity')['Price per Unit (Silver Drachma/kg)'].shift(lag)
    for lead in [1,2,3,4]:
        df[f'lead_{lead}'] = df.groupby('Commodity')['Price per Unit (Silver Drachma/kg)'].shift(-lead)

    lag_cols  = [f'lag_{l}'  for l in [1,2,3,4]]
    lead_cols = [f'lead_{l}' for l in [1,2,3,4]]
    df_clean = df.dropna(subset=lag_cols + lead_cols + ['commodity_enc'])

    X = df_clean[['commodity_enc'] + lag_cols]
    y = df_clean[lead_cols]
    return X, y, le

In [10]:
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

# Loop over all region–type combinations
regions = price_df['Region'].unique()
types   = price_df['Type'].unique()

results = []
for region in regions:
    for typ in types:
        X_train, y_train, le = prepare_dataset(price_df, region, typ, fit_encoder=True)
        X_eval,  y_eval,  _  = prepare_dataset(eval_price_df, region, typ, le=le, fit_encoder=False)

        if X_train.shape[0] < 10 or X_eval.shape[0] < 1:
            continue

        model = MultiOutputRegressor(
            XGBRegressor(objective='reg:squarederror', n_estimators=1000)
        )
        model.fit(X_train, y_train)

        # Save model and label encoder
        filename = f"{region}__{typ}.joblib".replace(" ", "_")
        joblib.dump({'model': model, 'label_encoder': le}, os.path.join(model_dir, filename))

        y_pred = model.predict(X_eval)
        mae   = mean_absolute_error(y_eval, y_pred)
        rmse  = mean_squared_error(y_eval, y_pred)**0.5

        results.append({
            'Region': region,
            'Type': typ,
            'TrainRows': X_train.shape[0],
            'EvalRows': X_eval.shape[0],
            'MAE': mae,
            'RMSE': rmse
        })

perf_df = pd.DataFrame(results)
print(perf_df)
print(f"\nAverage RMSE across all 50 models: {perf_df['RMSE'].mean():.4f}")

           Region       Type  TrainRows  EvalRows       MAE      RMSE
0    Mystic Falls      Fruit       3499       388  0.100715  1.767338
1    Mystic Falls  Vegetable       3475       364  0.000406  0.000546
2         Olympus      Fruit       3498       388  0.562811  9.908588
3         Olympus  Vegetable       3474       364  0.000385  0.000536
4      Metropolis      Fruit       3498       388  0.211659  3.722467
5      Metropolis  Vegetable       3474       364  0.000363  0.000506
6         Arcadia      Fruit       3498       388  0.121709  2.137738
7         Arcadia  Vegetable       3474       364  0.000382  0.000530
8         Elysium      Fruit       3498       388  0.116115  2.038800
9         Elysium  Vegetable       3474       364  0.000347  0.000476
10     Shangri-La      Fruit       3499       387  0.539968  9.494014
11     Shangri-La  Vegetable       3475       363  0.000389  0.000533
12       Valhalla      Fruit       3499       387  0.256003  4.498231
13       Valhalla  V

In [12]:
# Zip the folder
shutil.make_archive('/content/models', 'zip', '/content/models')

# Download the zipped file
files.download('/content/models.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# Path to your FastAPI-compatible SQLite DB
db_path = "/content/agroprophet.db"

# Step 1: Create DB with required tables if not exists
def create_tables_if_needed():
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        # Price table with new 'actual' column
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS price (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT NOT NULL,
                region TEXT NOT NULL,
                crop TEXT NOT NULL,
                price REAL NOT NULL,
                actual BOOLEAN NOT NULL DEFAULT 1
            )
        """)
        # Weather table (unchanged)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS weather (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT NOT NULL,
                region TEXT NOT NULL,
                rainfall REAL,
                humidity REAL,
                temp REAL
            )
        """)
        conn.commit()

create_tables_if_needed()
print("✅ Tables created (if not already present).")

# Step 2: Load and clean price_df, add 'actual' column
price_df_renamed = price_df.rename(columns={
    "Date": "date",
    "Region": "region",
    "Commodity": "crop",
    "Price per Unit (Silver Drachma/kg)": "price"
})
price_df_filtered = price_df_renamed[["date", "region", "crop", "price"]]
price_df_filtered["actual"] = True  # New column with value True for all rows

# Step 3: Load and clean weather_df
weather_df_renamed = weather_df.rename(columns={
    "Date": "date",
    "Region": "region",
    "Rainfall (mm)": "rainfall",
    "Humidity (%)": "humidity",
    "Temperature (K)": "temp"
})
weather_df_filtered = weather_df_renamed[["date", "region", "rainfall", "humidity", "temp"]]

# Step 4: Insert into database
with sqlite3.connect(db_path) as conn:
    price_df_filtered.to_sql("price", conn, if_exists="append", index=False)
    weather_df_filtered.to_sql("weather", conn, if_exists="append", index=False)

print("✅ All records inserted into both price and weather tables.")

✅ Tables created (if not already present).
✅ All records inserted into both price and weather tables.
